In [47]:
target_feature = 'vnf cpu usage'

In [101]:
import numpy as np
import pandas as pd
from multiprocessing.pool import ThreadPool
import csv
import matplotlib.pyplot as plt
import argparse
import time
import torch
import torch.nn as nn
# import preprocess_data
# from model import model
from torch import optim
from pathlib import Path
import pickle
# from anomalyDetector import fit_norm_distribution_param




import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation
import lightgbm as lgb
from scipy.stats import pearsonr





%matplotlib inline

In [11]:
fgs = pd.read_csv('fgs.csv')

In [12]:
fgs.head()

,,,,4963594775,1,2
proxy.9.5698,traffic counter.7.5664,0.971442,82.243099,1.794354,9.341767,100.0
5402488769,5,1.000000,NaN,NaN,NaN,NaN
traffic counter.7.7413,proxy.9.11244,4.060322,119.086081,16.548350,8.474562,150.0
proxy.9.11244,proxy.9.2362,0.268318,23.865256,11.573841,9.248256,150.0
proxy.9.2362,traffic counter.7.9524,2.961259,80.083566,18.280812,4.783467,150.0


In [13]:
vns = pd.read_csv('vnfs.csv', header=None)

In [30]:
vnf_id = vns[vns.columns[0]].values
vnf_type = vns[vns.columns[1]].values
vnf_scheduling = vns[vns.columns[2]].values
vnf_pm = vns[vns.columns[3]].values
vnf_fg = vns[vns.columns[4]].values
flavor_data = vns[vns.columns[5:8]].values
vm_data = vns[vns.columns[8:11]].values
usage_data = vns[vns.columns[11:15]].values

In [44]:
#Combining all the input data

input_data = ['vnf_type', 'vnf_scheduling', 'vnf_pm', 'vnf_fg', 'flavor_data', 'vm_data', 'usage_data']

In [35]:
for i in input_data:
    print(i, (eval(i)).dtype, eval(i).shape) #Exactly what we want

vnf_type int64 (12322,)
vnf_scheduling int64 (12322,)
vnf_pm int64 (12322,)
vnf_fg int64 (12322,)
flavor_data float64 (12322, 3)
vm_data float64 (12322, 3)
usage_data float64 (12322, 4)


In [41]:
data = pd.DataFrame()
for i in input_data:
    data = pd.concat([data, pd.DataFrame(eval(i))], axis=1)

In [43]:
# The reason we're doing this directly not taking this from the dataframe imported above, is because we just wanted to select specific rows. 
data.shape

(12322, 14)

In [95]:
output = pd.read_csv('new_results_out.csv', sep =';')#header=None)
output.drop('Unnamed: 0',axis = 1, inplace=True)
output = output[target_feature][:data.shape[0]].values

In [96]:
output.shape

(12322,)

### Now starting construction of model

In [97]:
lgbb = lgb.LGBMRegressor()

In [98]:
lgbb.fit(data, output)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [99]:
predictions = lgbb.predict(data)

In [100]:
predictions

array([29.25140092, 18.6289052 , 16.95407951, ..., 31.50262267,
       24.8838422 , 22.91102014])

In [102]:
pearsonr(predictions, output)

(0.6444400232833617, 0.0)

In [69]:
np.min(output)

0.0055320000000000005